In [1]:
import config
from pathlib import Path
OUTPUT_DIR = Path(config.OUTPUT_DIR)

In [2]:
import numpy as np
from matplotlib import pyplot as plt

In [6]:
from pathlib import Path

import numpy as np
import pandas as pd
import wrds

from load_CRSP_fund import load_CRSP_combined_file
from load_s12 import load_s12_file
from load_mflink import load_mflink1, load_mflink2


import config
WRDS_USERNAME = config.WRDS_USERNAME

df_crsp = load_CRSP_combined_file()
df_s12 = load_s12_file()
df_mflink1 = load_mflink1()
df_mflink2 = load_mflink2()

In [5]:
df_crsp

,caldt,crsp_fundno,mtna,mret,mnav,lipper_asset_cd,lipper_class_name,crsp_obj_cd
0,1997-12-31,2.0,72.000,0.012139,17.0900,None,None,EDYG
1,1998-01-30,2.0,82.409,0.009947,17.2600,None,None,EDYG
2,1998-02-27,2.0,92.030,0.043453,18.0100,None,None,EDYG
3,1998-03-31,2.0,105.852,0.031649,18.5800,None,None,EDYG
4,1998-04-30,2.0,112.308,0.004844,18.6700,None,None,EDYG
...,...,...,...,...,...,...,...,...
55783,2023-12-29,102099.0,0.500,NaN,25.1400,EQ,Small-Cap Core Funds,EDCS
55784,2023-12-29,102109.0,1.000,NaN,20.8119,EQ,Large-Cap Core Funds,EDYB
55785,2023-12-29,102110.0,0.100,NaN,10.0000,EQ,Large-Cap Growth Funds,EDYG
55786,2023-12-29,102111.0,2.000,NaN,10.0000,EQ,Large-Cap Growth Funds,EDYG


In [ ]:
"AHHHHH"